In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from diffusion_models import Sampler
from diffusion_models.denoising_models import dl_models
from diffusion_models.noise_schedulers import cosine_schedule, sigmoid_noiseschedule



#beta_min, beta_max = 1e-3, 0.2
#T = 100
#beta_t_cos, alpha_t_cos, alphabar_t_cos = cosine_schedule(T) 


T=1000
ns = "Sigmoid"
beta_t, alpha_t, alphabar_t = sigmoid_noiseschedule(T, start=-6, end=6, tau=0.7, clip_min=1e-9)



#denoiser = dl_models.Transformer(n_steps=96, input_dim=1, T=100)
model = "LSTM"
num_layers = 3
hdim = 512
norm = True

#checkpoint_dir = f"./outputs/checkpoints/Transformer/3-l/64-hd/"
#ckpt = 100



checkpoint_dir = f"./outputs/checkpoints/{model}/{num_layers}-l/{hdim}-hd/{T}-T/norm-{norm}/schedul-{ns}"
ckpt = sorted([ int(i[11:-4]) for i in os.listdir(checkpoint_dir)])[-1]
ckpt_dir = os.path.join(checkpoint_dir, f"checkpoint_{ckpt}.pth")
print(ckpt_dir)

denoiser = torch.load(ckpt_dir)

./outputs/checkpoints/LSTM/3-l/512-hd/1000-T/norm-True/schedul-Sigmoid/checkpoint_21.pth


In [ ]:
sampling_number = 32
only_final = False
shape = (96, 1)
device = "cpu"

process = Sampler(beta_t, alpha_t, alphabar_t, denoiser, device, shape=shape)
sample = process.sampling(sampling_number, only_final)
#scatter(sample[9::30], only_final)

sample = sample.detach().cpu().numpy()


In [ ]:

import matplotlib.pyplot as plt

# Create a figure and a set of subplots  
fig, ax = plt.subplots()  
  
# This will create a line for each row in your data. If that's too many lines,  
# you might need to select a subset of rows to plot.  
for i in range(sample.shape[1]):  
    ax.plot(sample[99, i], alpha=0.3)
    
    
plt.show()  

In [ ]:
from celluloid import Camera    

fig, ax = plt.subplots()  
camera = Camera(fig)  
for t in [i for i in range(0, 91, 10)]+[i for i in range(91, 100)]:
    for s in range(sample.shape[1]):
        time_sample_t = sample[t, s]
        ax.plot(time_sample_t, alpha=0.3)  
    
    # Add text showing frequency  
    text = ax.text(0.8, 0.8, f'Time: {100 - t}', transform=ax.transAxes) 
    camera.snap()  
    
    
animation = camera.animate()  
  
# Save as GIF  
animation.save(f'./outputs/denoising_{ns}_{T}.gif', writer='pillow')  
  
plt.show()  


In [ ]:
from IPython.display import Image  
Image(filename=f'./outputs/denoising_{ns}_{T}.gif')  


## Sampling 5000 generated consumption to compare with the real ones

In [ ]:
sampling_number = 5000
only_final = True
shape = (96, 1)
device = "cuda"

process = Sampler(beta_t_cos, alpha_t_cos, alphabar_t_cos, denoiser, device, shape=shape)
sample = process.sampling(sampling_number, only_final)
#scatter(sample[9::30], only_final)

sample = sample.detach().cpu().numpy()

np.save(f"./outputs/generated_consumption_{model}_{num}{}.npy", sample)